In [1]:
import requests
import json
import time
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell

    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data).text)                   
    end = time.time()                                                              
    print("Took: %s s" % (end - start))

    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [2]:
server = 'http://localhost:8001/jsoniq'

### 1.


In [3]:
%%rumble
count(
    for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
    where $i.guess eq $i.target and $i.target eq "Russian"
    return $i
)

Took: 15.837881565093994 s
290818


### 2.

In [4]:
%%rumble
let $tar := for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
return $i.target
return count(
    distinct-values($tar)
)

Took: 11.720677852630615 s
78


### 3.

In [5]:
%%rumble
for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
where $i.guess eq $i.target
order by $i.target ascending, $i.country ascending, $i.date ascending
count $c
where $c <= 2
return $i.sample

Took: 3.8221054077148438 s
"fdf23d0a7063ba2fcef4b18eb7d57ad8"
"efcd813daec1c836d9f030b30caa07ce"


### 4.

In [6]:
%%rumble
let $cnt := for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
group by $c:=$i.country, $t:=$i.target
return {"c": $c,
        "t": $t,
        "count": count(
            for $it in $i
            return $it.guess
        )}

for $item in $cnt
order by $item.count descending
count $c
where $c <= 3
return $item.count

Took: 5.058027505874634 s
1553
1464
1459


### 5.

In [7]:
%%rumble
round(
    count(
    for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
    where $i.guess eq $i.target and $i.guess eq $i.choices[[1]]
    return $i) div count(
        for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10) return $i), 4)

Took: 24.02294611930847 s
0.256


### 6.

In [8]:
%%rumble
let $perc := for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
group by $t:=$i.target
let $corr := count(for $cnt in $i where $cnt.target eq $cnt.guess return $cnt)
let $total := count(for $cnt in $i return $cnt)
return {"lang":$t, "percentage": $corr div $total}

for $it in $perc
order by $it.percentage descending
count $c
where $c <= 3
return $it.lang

Took: 4.494207143783569 s
"French"
"German"
"Mandarin"


### 7.

In [9]:
%%rumble
for $i in json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
group by $d:=$i.date
order by $d descending
count $c
where $c <= 1
return count($i)

Took: 20.544063568115234 s
65653
